In [2]:
# To install Bayes.jl, run:
# Pkg.clone("http://github.com/luiarthur/Bayes.jl")
using PlotlyJS, Distributions, Bayes

In [3]:
function mh(;B=10000, burn=Int(B*.8), m=0, s=10, a=3, b=3, cs_mu=1, cs_sig2=1)
    function logLike(mu, sig2)
        sig = sqrt(sig2)
        F(x) = cdf(Normal(mu,sig),x)
        logF(x) = logcdf(Normal(mu,sig),x)
        logS(x) = logccdf(Normal(mu,sig),x)
        
        14*logF(66) + 30*log(F(68)-F(66)) + 49*log(F(70)-F(68)) + 
        70*log(F(72)-F(70)) + 33*log(F(74)-F(72)) + 15*logS(74)
    end
    llpp_mu(mu,s2) = logLike(mu,s2) + logpdf(Normal(m,s), mu)
    llpp_sig2(mu,s2) = logLike(mu,s2) + logpdf(InverseGamma(a,b), s2)
    
    post = zeros(2,B)
    post[1,:] += m
    post[2,:] += s
    acc_mu = 0
    acc_sig2 = 0

    for i in 2:B
        # update mu
        cand_mu = randn() * cs_mu + post[1,i-1]
        log_ratio_mu = llpp_mu(cand_mu, post[2,i-1]) - llpp_mu(post[1,i-1], post[2,i-1])
        if log_ratio_mu > log(rand())
            post[1,i] = cand_mu
            i > burn ? acc_mu += 1 : 0
        else
            post[1,i] = post[1,i-1]
        end
  
        # update sig2
        cand_sig2 = randn() * cs_sig2 + post[2,i-1]
        if cand_sig2 > 0
            log_ratio_sig2 = llpp_sig2(post[1,i], cand_sig2) - llpp_sig2(post[1,i],post[2,i-1])
            if log_ratio_sig2 > log(rand())
                post[2,i] = cand_sig2
                i > burn ? acc_sig2 += 1 : 0
            else
                post[2,i] = post[2,i-1]
            end
        else
            post[2,i] = post[2,i-1]
        end
    end

    println("acceptance μ: ",acc_mu/(B-burn))
    println("acceptance σ²: ",acc_sig2/(B-burn))
    println()
    (post[:,burn:end]',acc_mu/B,acc_sig2/B)
end

mh (generic function with 1 method)

### The Inverse Gamma in Julia corresponds to parameterization where the beta parameter is in the numerator of the exponent. i.e. 

$$
    \large f(x; \alpha, \beta) = \frac{\beta^\alpha x^{-(\alpha + 1)}}{\Gamma(\alpha)}
    e^{-\frac{\beta}{x}}, \quad x > 0
$$

In [4]:
function auxGibbs(;B=10000, burn=Int(B*.8), m=80, s=10, a=3, b=3)
    post = zeros(2,B)
    post[1,:] = m
    post[2,:] = s
    counts = [14, 30, 49, 70, 33, 15]
    N = sum(counts)
    cuts = [64, 66, 68, 70, 72, 74, 100]
    z = []
    for i in 1:length(counts), j in 1:counts[i]
        push!(z,cuts[i])
    end
    inds = [cuts[i-1] .<= z .< cuts[i] for i in 2:length(cuts)]
    n = [sum(ind) for ind in inds]
    
    updateSig2(mu,z) = rand(InverseGamma( a+N/2, b+sum((z-mu).^2)/2) )
    function updateMu(sig2,z)
        s2_new = (sig2*s^2) / (sig2+N*s^2)
        mu_new = (N*s^2*mean(z) + sig2*m) / (sig2+N*s^2)
        randn() * sqrt(s2_new) + mu_new
    end
    function updateZ(z,mu,sig2)
        rtN(lower,upper,n) =  rand(TruncatedNormal(mu,sqrt(sig2),lower,upper),n)
        for i in 2:length(cuts)
            #ind = cuts[i-1] .<= z .< cuts[i]
            #ind = inds[i-1]
            #n = sum(ind)
            z[inds[i-1]] = rtN(cuts[i-1],cuts[i],n[i-1])
        end
        z
    end
    
    for i in 2:B
        post[1,i] = updateMu(post[2,i-1],z)
        post[2,i] = updateSig2(post[1,i],z)
        #println(post[:,i])
        #println("update z iteration: ", i)
        z = updateZ(z,post[1,i],post[2,i])
        #println(z)
    end
    
    post[:,burn:end]'
end

auxGibbs (generic function with 1 method)

In [5]:
@time post = mh(B=100000,burn=90000,m=100,s=10,a=3,b=1,cs_mu=.5,cs_sig2=3);
post_mu = collect(post[1][:,1])
post_s2 = collect(post[1][:,2]);

acceptance μ: 0.4062
acceptance σ²: 0.2963

  2.221108 seconds (31.24 M allocations: 1.081 GB, 9.57% gc time)


In [6]:
function draw(m,s,t)
    p1=scatter(y=m,x=1:length(post_mu),mode="lines",marker=attr(color=:grey,size=3))
    l1=Layout(yaxis_title="mu")
    p2=scatter(y=s,x=1:length(post_s2),mode="lines",marker=attr(color=:grey,size=3))
    l2=Layout(yaxis_title="sig2")
    p = [plot(p1,l1), plot(p2,l2)]
    p.plot.layout["showlegend"] = false
    p.plot.layout["title"] = t
    p
end

draw (generic function with 1 method)

In [7]:
draw(post_mu,post_s2,"Posterior Trace Plots (Metropolis Hastings)")

In [8]:
Bayes.plotposts(post[1],names=["μ","σ²"],spacing=.01)

In [9]:
@time post_ag = auxGibbs(B=100000,burn=90000)
post_mu_ag = collect(post_ag[:,1])
post_s2_ag = collect(post_ag[:,2]);

  4.924717 seconds (131.11 M allocations: 2.568 GB, 4.76% gc time)


In [10]:
draw(post_mu_ag,post_s2_ag,"Posterior Trace Plots (Auxiliary Gibbs)")

In [11]:
# For reference, this is how you plot a histogram:
# p11 = plot(histogram(x=post_mu_ag,marker=attr(color=:orange),histnorm=:probability),Layout(yaxis_zeroline=false))
# p11.plot.layout["bargap"] = .1
# p11

Bayes.plotposts(post_ag,names=["μ","σ²"],spacing=.01)

In [12]:
println(mean(post[1],1))
println(mean(post_ag,1))
println()
println(cov(post[1]))
println(cov(post_ag))

[70.17423660757336 6.879582257526463]
[70.19624998095948 6.567068383524845]

[0.03475509871638922 0.0004465589484980783
 0.0004465589484980783 0.593470536179501]
[0.033139755683620736 0.002857008369643332
 0.002857008369643332 0.5032369887792979]


In [13]:
println(hpd(post[1][:,1]))
println(hpd(post_ag[:,1]))

println(hpd(post[1][:,2]))
println(hpd(post_ag[:,2]))

[69.81886549625268,70.54535413297094]
[69.8376624468251,70.55099705016931]
[5.518710319936187,8.468543309239148]
[5.23154354456001,7.986461806652989]
